In [ ]:
!pip install --upgrade transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 61.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


In [6]:
!pip install transformers

In [22]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification


class TextDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=256):
        self.texts = (df["utterance"]+" - "+df["context"]).tolist()
        self.labels = df["label"].astype("category")
        self.label_ids = self.labels.cat.codes
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label_map = dict(enumerate(self.labels.cat.categories))

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.label_ids.iloc[idx], dtype=torch.long)
        return item


df = pd.read_csv("subset_labeled.tsv", sep="\t")
print("file read\n")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42,
                                     stratify=df["label"])
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
train_ds = TextDataset(train_df, tokenizer)
test_ds = TextDataset(test_df, tokenizer)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=8)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(train_df["label"].unique()),
    problem_type="single_label_classification"
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
for epoch in tqdm(range(30)):
    model.train()
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"[Epoch {epoch+1}] Loss = {loss.item():.4f}")
model.eval()
preds, gold = [], []

with torch.no_grad():
    for batch in test_loader:
        labels = batch["labels"]
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        logits = outputs.logits.cpu()
        preds.extend(logits.argmax(dim=1).tolist())
        gold.extend(labels.tolist())

label_names = train_df["label"].astype("category").cat.categories
print(classification_report(gold, preds, labels=range(len(label_names)), target_names=label_names))

model.save_pretrained("semantic_classifier_distilbert")
tokenizer.save_pretrained("semantic_classifier_distilbert")

file read



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  3%|▎         | 1/30 [00:01<00:54,  1.88s/it]

[Epoch 1] Loss = 1.8904


  7%|▋         | 2/30 [00:03<00:50,  1.81s/it]

[Epoch 2] Loss = 1.7094


...

[Epoch 29] Loss = 0.0068


100%|██████████| 30/30 [00:55<00:00,  1.84s/it]

[Epoch 30] Loss = 0.0157
              precision    recall  f1-score   support

         AGG       0.20      0.20      0.20         5
        ARTH       0.17      0.33      0.22         3
        COMP       0.33      0.33      0.33         3
      LOOKUP       0.50      0.50      0.50         4
        Next       0.00      0.00      0.00         1
       SUPER       1.00      0.50      0.67         4
       other       0.00      0.00      0.00         0

    accuracy                           0.35        20
   macro avg       0.31      0.27      0.27        20
weighted avg       0.42      0.35      0.37        20




/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/m

('semantic_classifier_distilbert/tokenizer_config.json',
 'semantic_classifier_distilbert/special_tokens_map.json',
 'semantic_classifier_distilbert/vocab.txt',
 'semantic_classifier_distilbert/added_tokens.json',
 'semantic_classifier_distilbert/tokenizer.json')